In [1]:
import os
import torch
import librosa

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from datasets import AudioDataset

import sklearn.metrics as met
# hyper parameters
MAX_EPOCH = 100

/home/wew016/.local/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/wew016/.local/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
# path to urban sound 8k
data_root = "/home/wew016/UrbanSound8K/"
# path to label
label_path = "/home/wew016/UrbanSound8K/metadata/UrbanSound8K.csv"        

In [3]:
# initialize dataset (feature can be "mfcc" or "spec")
audio_dataset = AudioDataset(10, DataRoot=data_root, LabelPath=label_path, feature="mel_mean_db", mode="train")

verify mel_mean_db feature success


In [4]:
# initialize dataloader
data_loader = torch.utils.data.DataLoader(audio_dataset, batch_size=len(audio_dataset), shuffle=True, num_workers=1)

for idx, data in enumerate(data_loader):
        x_train, y_train = data
        print(x_train.shape)
        print(y_train.shape) 

torch.Size([7895, 128])
torch.Size([7895])


In [5]:
x_train=x_train.numpy()
x_train.shape

(7895, 128)

In [6]:
y_train=y_train.numpy()
y_train.shape

(7895,)

In [7]:
audio_dataset_test = AudioDataset(10, DataRoot=data_root, LabelPath=label_path, feature="mel_mean_db", mode="test")

verify mel_mean_db feature success


In [8]:
# initialize dataloader
data_loader_test = torch.utils.data.DataLoader(audio_dataset_test, batch_size=len(audio_dataset_test), shuffle=True, num_workers=1)

for idx, data in enumerate(data_loader_test):
        x_test, y_test = data
        print(x_test.shape)
        print(y_test.shape)    

torch.Size([837, 128])
torch.Size([837])


In [9]:
x_test=x_test.numpy()
x_test.shape

(837, 128)

In [10]:
y_test=y_test.numpy()
y_test.shape

(837,)

<font color=red size=5>
XGBOOST Model
</font>

In [11]:
from xgboost import XGBClassifier

In [12]:
model_xgboost = XGBClassifier(learning_rate =0.2, 
                      n_estimators=150, 
                      max_depth=4,
                      num_class=10,    
                      min_child_weight=5, 
                      gamma=0.1, 
                      subsample=0.5, 
                      colsample_bytree=0.5,
                      objective='multi:softmax', 
                      seed=50)

In [13]:
model_xgboost.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=4,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=150, n_jobs=0, num_class=10, num_parallel_tree=1,
              objective='multi:softprob', random_state=50, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=50, subsample=0.5,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
pred_train_xgboost = model_xgboost.predict(x_train)
met.accuracy_score(y_train,pred_train_xgboost)

0.9911336288790373

In [15]:
pred_test_xgboost = model_xgboost.predict(x_test)

In [16]:
y_test

array([8, 5, 5, 2, 3, 3, 1, 0, 4, 5, 9, 3, 9, 2, 7, 7, 3, 3, 8, 6, 4, 7,
       1, 9, 3, 3, 8, 5, 4, 2, 3, 0, 7, 7, 0, 3, 7, 1, 7, 9, 8, 9, 3, 7,
       7, 4, 2, 9, 8, 8, 7, 4, 8, 5, 7, 4, 0, 5, 9, 2, 4, 3, 6, 4, 3, 5,
       7, 9, 1, 4, 2, 3, 4, 2, 2, 7, 3, 3, 9, 8, 7, 2, 4, 6, 4, 0, 0, 3,
       7, 3, 9, 8, 3, 8, 4, 5, 5, 8, 5, 4, 8, 9, 5, 2, 7, 3, 7, 0, 4, 0,
       8, 8, 9, 3, 5, 5, 2, 2, 4, 5, 2, 2, 3, 9, 7, 7, 0, 4, 4, 4, 3, 5,
       0, 0, 9, 2, 3, 7, 1, 5, 5, 5, 4, 5, 3, 3, 3, 4, 7, 6, 4, 3, 0, 7,
       8, 0, 1, 3, 0, 7, 5, 5, 4, 8, 7, 1, 4, 6, 4, 1, 3, 9, 8, 8, 3, 5,
       2, 2, 7, 9, 5, 2, 0, 1, 2, 5, 0, 4, 7, 0, 4, 0, 8, 4, 9, 7, 9, 2,
       5, 7, 5, 9, 0, 5, 9, 2, 0, 7, 0, 0, 9, 6, 7, 0, 8, 7, 6, 2, 9, 5,
       4, 3, 2, 2, 2, 4, 8, 0, 5, 3, 5, 1, 7, 8, 3, 3, 2, 2, 9, 3, 0, 4,
       2, 5, 0, 4, 4, 9, 5, 7, 7, 2, 2, 2, 0, 5, 4, 8, 8, 4, 7, 3, 4, 9,
       4, 9, 8, 4, 0, 8, 0, 7, 8, 0, 3, 3, 6, 5, 9, 5, 9, 0, 2, 7, 7, 2,
       7, 9, 0, 3, 9, 7, 7, 3, 5, 2, 7, 2, 4, 7, 2,

In [17]:
pred_test_xgboost

array([8, 5, 3, 2, 3, 3, 1, 0, 4, 5, 1, 3, 9, 3, 2, 7, 3, 3, 0, 3, 4, 4,
       1, 9, 4, 8, 0, 3, 4, 9, 3, 0, 4, 4, 0, 3, 4, 1, 4, 2, 8, 9, 4, 4,
       4, 7, 2, 9, 0, 8, 7, 4, 2, 0, 7, 5, 2, 5, 9, 2, 5, 7, 6, 7, 2, 5,
       4, 4, 1, 4, 2, 3, 7, 2, 2, 7, 3, 3, 9, 1, 2, 8, 8, 8, 4, 7, 2, 2,
       7, 3, 6, 9, 3, 5, 7, 2, 9, 2, 5, 4, 9, 9, 5, 2, 7, 3, 4, 0, 4, 0,
       2, 4, 2, 3, 5, 7, 2, 3, 4, 2, 9, 2, 3, 2, 7, 7, 0, 4, 4, 8, 3, 5,
       0, 0, 2, 2, 3, 7, 1, 5, 2, 5, 8, 5, 3, 9, 5, 7, 7, 3, 4, 3, 7, 7,
       8, 0, 1, 3, 0, 4, 5, 3, 4, 2, 7, 1, 7, 9, 4, 1, 2, 2, 3, 8, 3, 5,
       2, 2, 7, 9, 2, 2, 0, 1, 2, 5, 0, 4, 7, 0, 7, 0, 9, 4, 1, 4, 9, 2,
       5, 7, 9, 9, 7, 5, 9, 2, 0, 4, 0, 0, 9, 6, 4, 0, 8, 7, 6, 2, 9, 2,
       4, 3, 2, 8, 8, 2, 2, 0, 5, 3, 5, 1, 7, 8, 3, 2, 2, 2, 4, 3, 9, 4,
       9, 5, 3, 7, 4, 9, 5, 4, 2, 2, 9, 3, 7, 5, 4, 9, 8, 8, 4, 3, 7, 9,
       9, 2, 8, 4, 7, 2, 2, 4, 8, 2, 3, 9, 3, 1, 2, 5, 9, 0, 2, 4, 4, 2,
       4, 8, 7, 4, 9, 4, 4, 3, 5, 8, 7, 3, 0, 7, 2,

In [18]:
met.accuracy_score(y_test,pred_test_xgboost)

0.5722819593787336

In [19]:
met.confusion_matrix(y_test,pred_test_xgboost)

array([[59,  0, 21,  1,  0,  0,  0, 13,  1,  5],
       [ 1, 24,  2,  0,  0,  0,  1,  1,  0,  4],
       [ 0,  0, 64, 15,  0,  2,  1,  0,  8, 10],
       [ 0,  0, 11, 66,  5,  3,  1,  3,  2,  9],
       [ 1,  0,  6,  4, 50,  5,  0, 26,  6,  2],
       [ 1,  1, 15,  7,  2, 53,  0,  5,  0,  9],
       [ 0,  0,  6,  7,  0,  0, 16,  0,  2,  1],
       [ 3,  0,  6,  0, 44,  0,  0, 43,  0,  0],
       [ 4,  1, 18,  6,  1,  1,  0,  1, 36, 15],
       [ 2,  2, 17,  2,  5,  0,  1,  0,  3, 68]])

<font color=red size=5>
SVM Model
</font>

In [20]:
import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
svr = SVC()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 2, 4], 'gamma':[0.125, 0.25, 0.5 ,1, 2, 4]}
clf = GridSearchCV(svr, parameters, scoring='f1_samples')
clf.fit(x_train, y_train)
print('The parameters of the best model are: ')
print(clf.best_params_)

In [22]:
model_svm = sklearn.svm.SVC(C=0.75, 
                            kernel='rbf', 
                            degree=3, 
                            gamma='auto', 
                            coef0=0.0, 
                            shrinking=True, 
                            probability=False, 
                            tol=0.001, 
                            cache_size=200, 
                            class_weight=None, 
                            verbose=False, 
                            max_iter=-1, 
                            decision_function_shape=None, 
                            random_state=None)

In [23]:
model_svm.fit(x_train,y_train)

SVC(C=0.75, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [24]:
pred_train_svm = model_svm.predict(x_train)
met.accuracy_score(y_train,pred_train_svm)

0.9982267257758075

In [25]:
pred_test_svm = model_svm.predict(x_test)
met.accuracy_score(y_test,pred_test_svm)

0.15292712066905614

<font color=red size=5>
RandomForest Model
</font>

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
model_RF = RandomForestClassifier(n_estimators=100 ,criterion = "entropy", bootstrap= False)

In [28]:
model_RF.fit(x_train,y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
pred_train_RF = model_RF.predict(x_train)
met.accuracy_score(y_train,pred_train_RF)

1.0

In [30]:
pred_test_RF = model_RF.predict(x_test)
met.accuracy_score(y_test,pred_test_RF)

0.5185185185185185

<font color=red size=5>
KNN Model
</font>

In [31]:
from sklearn.neighbors import KNeighborsClassifier

In [32]:
model_KNN = KNeighborsClassifier(n_neighbors=10, 
                                 weights='uniform',
                                 algorithm='auto',
                                 leaf_size=30, 
                                 p=2, 
                                 metric='minkowski', 
                                 metric_params=None, 
                                 n_jobs=None)

In [33]:
model_KNN.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [34]:
pred_train_KNN = model_KNN.predict(x_train)
met.accuracy_score(y_train,pred_train_KNN)

0.8379987333755542

In [35]:
pred_test_KNN = model_KNN.predict(x_test)
met.accuracy_score(y_test,pred_test_KNN)

0.3906810035842294

<font color=red size=5>
KNN Model
</font>